Code expects `materials.parquet` in the folder.  This was created using Excel given by Rejane

In [1]:

# !pip install requests, tqdm, duckdb

In [2]:
import requests
import duckdb
import re
from tqdm import tqdm


In [3]:
conn = duckdb.connect('consolidado.db')
conn.install_extension('spatial')
conn.load_extension('spatial')
for year in tqdm(range(2015, 2023)):
    attempts = 0
    max_attempts = 3
    success = False

    while attempts < max_attempts and not success:
        try:
            url = f'https://www.gov.br/saude/pt-br/acesso-a-informacao/banco-de-precos/bases-anuais-compiladas/registro-de-compras-compilado-ano-base-{year}/@@download'
            response = requests.get(url)

            if response.status_code == 200:
                with open(f'registros_{year}.xlsx', 'wb') as file:
                    file.write(response.content)

                query = f"CREATE TABLE t{year} AS SELECT * FROM st_read('registros_{year}.xlsx');"
                conn.execute(query)
                success = True
            else:
                print(f"Attempt {attempts + 1} failed for year {year}: status code {response.status_code}")

        except Exception as e:
            print(f"Error on attempt {attempts + 1} for year {year}: {e}")

        attempts += 1

    if not success:
        print(f"Failed to process year {year} after {max_attempts} attempts.")
        # Handle the failure case here, e.g., logging, raising an exception, etc.

conn.close()# Rest of your code...


100%|██████████| 8/8 [00:43<00:00,  5.40s/it]


In [4]:
conn = duckdb.connect('consolidado.db')
# Fetch the list of all table names
table_names = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()
table_names = [table[0] for table in table_names]

# Function to remove non A-Z characters and append a unique number
def clean_column_name(old_name, index):
    clean_name = re.sub(r'[^A-Z]', '', old_name.upper())
    return f"{clean_name}_{index}"

# Iterate over each table and rename its columns
for table in table_names:
    # Fetch the column names of the table
    column_names = conn.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table}' ORDER BY ordinal_position").fetchall()
    column_names = [col[0] for col in column_names]

    # Generate new column names
    new_column_names = [clean_column_name(name, idx) for idx, name in enumerate(column_names, start=1)]

    # Generate and execute ALTER TABLE statements for each column
    for old_name, new_name in zip(column_names, new_column_names):
        alter_statement = f'ALTER TABLE {table} RENAME COLUMN "{old_name}" TO {new_name};'
        conn.execute(alter_statement)
conn.close()

In [5]:
conn = duckdb.connect('consolidado.db')

conn.execute("""
    DROP TABLE IF EXISTS REGISTROS;
    CREATE TABLE REGISTROS AS (
    FROM t2015 SELECT
    CDIGOBR_1 AS codigo,
    DESCRIOITEM_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    REGISTROANVISA_5 AS anvisa,
    DATACOMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPOCOMPRA_9 AS tipo,
    FABRICANTE_10 AS fabricante,
    CNPJFABRICANTE_11 AS cnpj_fabricante,
    FORNECEDOR_12 AS fornecedor,
    CNPJFORNECEDOR_13 AS cnpj_fornecedor,
    NOMEINSTITUIO_15 AS instituicao,
    CNPJINSTITUIO_16 AS cnpj_instituicao,
    MUNICPIOINSTITUIO_17 AS municipio,
    UF_19 AS UF,
    LICITAO_20 AS licitacao,
    NOTAFISCAL_21 AS NF,
    QTDITENSCOMPRADOS_23 AS qtde,
    PAGO_24 AS unitario)
;""")
conn.execute("""
    INSERT INTO REGISTROS
    SELECT
    CDIGOBR_1 AS codigo,
    DESCRIOITEM_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    REGISTROANVISA_5 AS anvisa,
    DATACOMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPOCOMPRA_9 AS tipo,
    FABRICANTE_10 AS fabricante,
    CNPJFABRICANTE_11 AS cnpj_fabricante,
    FORNECEDOR_12 AS fornecedor,
    CNPJFORNECEDOR_13 AS cnpj_fornecedor,
    NOMEINSTITUIO_15 AS instituicao,
    CNPJINSTITUIO_16 AS cnpj_instituicao,
    MUNICPIOINSTITUIO_17 AS municipio,
    UF_19 AS UF,
    LICITAO_20 AS licitacao,
    NOTAFISCAL_21 AS NF,
    QTDITENSCOMPRADOS_23 AS qtde,
    PREOUNITRIO_24 AS unitario
    FROM t2016
;""")
conn.execute("""
    INSERT INTO REGISTROS
    SELECT
    CDIGOBR_1 AS codigo,
    DESCRIOCATMAT_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    REGISTROANVISA_5 AS anvisa,
    DATACOMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPOCOMPRA_9 AS tipo,
    FABRICANTE_10 AS fabricante,
    CNPJFABRICANTE_11 AS cnpj_fabricante,
    FORNECEDOR_12 AS fornecedor,
    CNPJFORNECEDOR_13 AS cnpj_fornecedor,
    NOMEINSTITUIO_14 AS instituicao,
    CNPJINSTITUIO_15 AS cnpj_instituicao,
    MUNICPIOINSTITUIO_16 AS municipio,
    UF_17 AS UF,
    '' AS licitacao,
    '' AS NF,
    QTDITENSCOMPRADOS_18 AS qtde,
    PREOUNITRIO_19 AS unitario
    FROM t2017
;""")
conn.execute("""
    INSERT INTO REGISTROS
    SELECT
    CDIGOBR_1 AS codigo,
    DESCRIO_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    ANVISA_5 AS anvisa,
    COMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPO_9 AS tipo,
    NOMEFABRICANTE_11 AS fabricante,
    FABRICANTE_10 AS cnpj_fabricante,
    NOMEFORNECEDOR_13 AS fornecedor,
    FORNECEDOR_12 AS cnpj_fornecedor,
    NOMEDAINSTITUIO_15 AS instituicao,
    INSTITUIO_14 AS cnpj_instituicao,
    MUNICPIO_16 AS municipio,
    UF_17 AS UF,
    '' AS licitacao,
    '' AS NF,
    QUANTIDADE_18 AS qtde,
    UNITRIO_19 AS unitario
    FROM t2018
;""")
conn.execute("""
    INSERT INTO REGISTROS
    SELECT
    CDIGOBR_1 AS codigo,
    DESCRIO_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    ANVISA_5 AS anvisa,
    COMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPO_9 AS tipo,
    NOMEFABRICANTE_11 AS fabricante,
    FABRICANTE_10 AS cnpj_fabricante,
    NOMEFORNECEDOR_13 AS fornecedor,
    FORNECEDOR_12 AS cnpj_fornecedor,
    NOMEDAINSTITUIO_15 AS instituicao,
    INSTITUIO_14 AS cnpj_instituicao,
    MUNICPIO_16 AS municipio,
    UF_17 AS UF,
    '' AS licitacao,
    '' AS NF,
    QUANTIDADE_18 AS qtde,
    UNITRIO_19 AS unitario
    FROM t2019
;""")
conn.execute("""
    INSERT INTO REGISTROS
    SELECT
    CDIGOBR_1 AS codigo,
    DESCRIO_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    ANVISA_5 AS anvisa,
    COMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPO_9 AS tipo,
    NOMEDOFABRICANTE_11 AS fabricante,
    FABRICANTE_10 AS cnpj_fabricante,
    NOMEDOFORNECEDOR_13 AS fornecedor,
    FORNECEDOR_12 AS cnpj_fornecedor,
    NOMEDAINSTITUIO_15 AS instituicao,
    INSTITUIO_14 AS cnpj_instituicao,
    MUNICPIO_16 AS municipio,
    UF_17 AS UF,
    '' AS licitacao,
    '' AS NF,
    QUANTIDADE_18 AS qtde,
    UNITRIO_19 AS unitario
    FROM t2020
;""")
conn.execute("""
    INSERT INTO REGISTROS
    SELECT
    CDIGOBR_1 AS codigo,
    DESCRIOCATMAT_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    REGISTROANVISA_5 AS anvisa,
    DATACOMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPOCOMPRA_9 AS tipo,
    FABRICANTE_10 AS fabricante,
    CNPJFABRICANTE_11 AS cnpj_fabricante,
    FORNECEDOR_12 AS fornecedor,
    CNPJFORNECEDOR_13 AS cnpj_fornecedor,
    NOMEINSTITUIO_14 AS instituicao,
    CNPJINSTITUIO_15 AS cnpj_instituicao,
    MUNICPIOINSTITUIO_16 AS municipio,
    UF_17 AS UF,
    '' AS licitacao,
    '' AS NF,
    QTDITENSCOMPRADOS_18 AS qtde,
    PREOUNITRIO_19 AS unitario
    FROM t2021
;""")
conn.execute("""
    INSERT INTO REGISTROS
    SELECT
    CDIGOBR_1 AS codigo,
    DESCRIOCATMAT_2 AS descricao,
    UNIDADEDEFORNECIMENTO_3 AS unidade,
    ANVISA_5 AS anvisa,
    COMPRA_6 AS data,
    MODALIDADEDACOMPRA_7 AS modalidade,
    TIPOCOMPRA_9 AS tipo,
    FABRICANTE_10 AS fabricante,
    CNPJFABRICANTE_11 AS cnpj_fabricante,
    FORNECEDOR_12 AS fornecedor,
    CNPJFORNECEDOR_13 AS cnpj_fornecedor,
    NOMEINSTITUIO_14 AS instituicao,
    CNPJINSTITUIO_15 AS cnpj_instituicao,
    MUNICPIOINSTITUIO_16 AS municipio,
    UF_17 AS UF,
    '' AS licitacao,
    '' AS NF,
    QTDITENSCOMPRADOS_18 AS qtde,
    PREOUNITRIO_19 AS unitario
    FROM t2022
;""")
conn.close()

In [6]:
conn = duckdb.connect('consolidado.db')
tables_to_drop = ['t2015', 't2016', 't2017', 't2018', 't2019', 't2020', 't2021', 't2022']
for table in tables_to_drop:
    conn.execute(f"DROP TABLE IF EXISTS {table};")

conn.close()

In [7]:
conn = duckdb.connect('consolidado.db')
conn.execute("""
CREATE TABLE registros_temp AS
SELECT *, ROW_NUMBER() OVER () AS id
from registros;

ALTER TABLE registros_temp
ADD COLUMN valid_date DATE;
UPDATE registros_temp
SET valid_date = CASE
                    WHEN data LIKE '____-__-__' THEN
                        CAST(data AS DATE)
                    WHEN data LIKE '__/__/____' THEN
                        CAST(
                            SUBSTRING(data, 7, 4) || '-' ||
                            SUBSTRING(data, 4, 2) || '-' ||
                            SUBSTRING(data, 1, 2)
                        AS DATE)
                    ELSE
                        NULL
                END;
alter table registros_temp drop column data;
alter table registros_temp rename valid_date to data;
drop table REGISTROS;
ALTER TABLE registros_temp rename to REGISTROS;
;""")
conn.close()

In [8]:
conn = duckdb.connect('consolidado.db')
conn.execute("""
CREATE TABLE MATERIALS AS
SELECT *
FROM read_parquet('materials.parquet')
;
delete from materials where id=81699 or id=106624; -- duplicated
""")
conn.close()

In [9]:
conn = duckdb.connect('consolidado.db')
conn.execute("ALTER TABLE REGISTROS ADD COLUMN problema VARCHAR")
conn.execute("UPDATE REGISTROS SET problema = 'sem cnpj instituicao' WHERE cnpj_instituicao IS NULL")
conn.execute("UPDATE REGISTROS SET problema = 'sem cnpj fabricante' WHERE cnpj_fabricante is null and problema is null")
conn.execute("""
UPDATE registros SET problema = 'codigobr nao encontrado' WHERE problema is NULL and codigo NOT IN (SELECT DISTINCT codigobr FROM MATERIALS);
UPDATE registros SET problema = 'unidade nao encontrada' WHERE problema is NULL and unidade NOT IN (SELECT DISTINCT unidade FROM MATERIALS);
""")
conn.execute("""
    UPDATE registros
    SET tipo = CASE
        WHEN tipo = 'j' OR tipo = 'J' THEN 'J'
        ELSE 'A'
    END
""")
conn.execute("""
UPDATE REGISTROS
SET cnpj_instituicao = LPAD(REGEXP_REPLACE(cnpj_instituicao, '[^0-9]', '', 'g'), 14, '0')
WHERE cnpj_instituicao IS NOT NULL;

UPDATE REGISTROS
SET cnpj_fornecedor = LPAD(REGEXP_REPLACE(cnpj_fornecedor, '[^0-9]', '', 'g'), 14, '0')
WHERE cnpj_fornecedor IS NOT NULL;

UPDATE REGISTROS
SET cnpj_fabricante = LPAD(REGEXP_REPLACE(cnpj_fabricante, '[^0-9]', '', 'g'), 14, '0')
WHERE cnpj_fabricante IS NOT NULL;

""")
conn.execute("""
update REGISTROS
set municipio = 'MOGI MIRIM'
where municipio = 'MOJI MIRIM';
""")

conn.close()

In [10]:
conn = duckdb.connect('consolidado.db')
conn.execute("""
CREATE TABLE PJ AS
SELECT cnpj, nome, municipio, UF
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY cnpj ORDER BY municipio_null, UF_null) as rn
    FROM (
        SELECT cnpj_instituicao AS cnpj,
               instituicao AS nome,
               municipio,
               UF,
               CASE WHEN municipio IS NULL THEN 1 ELSE 0 END AS municipio_null,
               CASE WHEN UF IS NULL THEN 1 ELSE 0 END AS UF_null
        FROM registros
        WHERE cnpj_instituicao IS NOT NULL AND problema IS NULL
        UNION
        SELECT cnpj_fornecedor AS cnpj,
               fornecedor AS nome,
               NULL AS municipio,
               NULL AS UF,
               1 AS municipio_null,
               1 AS UF_null
        FROM registros
        WHERE cnpj_fornecedor IS NOT NULL AND problema IS NULL
        UNION
        SELECT cnpj_fabricante AS cnpj,
               fabricante AS nome,
               NULL AS municipio,
               NULL AS UF,
               1 AS municipio_null,
               1 AS UF_null
        FROM registros
        WHERE cnpj_fabricante IS NOT NULL AND problema IS NULL
    ) AS combined
) AS ranked
WHERE rn = 1;

""")
conn.execute("""
CREATE TABLE COMPRAS AS
select distinct cnpj_instituicao, data, modalidade, tipo
from REGISTROS where problema is null
;
""")
conn.execute("""
CREATE TABLE compras_temp AS
SELECT *, ROW_NUMBER() OVER () AS id
FROM compras;
DROP TABLE compras;
ALTER TABLE compras_temp RENAME TO COMPRAS;
""")
conn.close()


In [11]:
conn = duckdb.connect('consolidado.db')
conn.execute("""
create table ITENS as
 select
r.id, c.id as compra_id,
r.codigo, r.descricao, r.unidade, r.anvisa,
r.data, r.modalidade, r.tipo, r.fabricante, r.cnpj_fabricante,
r.fornecedor, r.cnpj_fornecedor, r.instituicao, r.cnpj_instituicao,
r.municipio, r.uf, r.licitacao, r.nf, r.qtde, r.unitario,
from (select * from registros where problema is null)as r
join compras as c
on
r.cnpj_instituicao=c.cnpj_instituicao and
r.data=c.data and
r.modalidade = c.modalidade and
r.tipo = c.tipo
""")

conn.execute("""
alter table itens drop column data;
alter table itens drop column modalidade;
alter table itens drop column tipo;
alter table itens drop column cnpj_instituicao;
alter table itens drop column instituicao;
alter table itens drop column fornecedor;
alter table itens drop column fabricante;
alter table itens drop column municipio;
alter table itens drop column uf;
""")

conn.execute("""
create table nI AS
(select itens.* exclude (codigo, unidade, descricao), materials.id as materials_id
 from itens left join materials
 on itens.codigo=materials.codigobr and itens.unidade = materials.unidade)
;
drop table ITENS;
alter table nI rename to ITENS;
             """)
conn.close()


In [12]:
conn = duckdb.connect('consolidado.db')


conn.execute("""
copy pj to 'pj.parquet' (FORMAT PARQUET);
copy compras to 'compras.parquet' (FORMAT PARQUET);
copy itens to 'itens.parquet' (FORMAT PARQUET);
""")
conn.close()

In [13]:
conn = duckdb.connect('consolidado.db')


conn.execute("""
CREATE VIEW ITENSCOMPRA AS
(SELECT
    itens.id,
    compras.data,
    compras.cnpj_instituicao,
    PJIN.nome as instituicao,
    PJIN.municipio,
    PJIN.UF,
    compras.modalidade,
    compras.tipo,
    itens.licitacao,
    itens.materials_id,
    itens.cnpj_fornecedor,
    PJFO.nome as fornecedor,
    itens.NF,
    itens.cnpj_fabricante,
    PJFA.nome as fabricante,
    itens.anvisa,
    itens.qtde,
    itens.unitario
FROM itens
LEFT JOIN compras ON itens.compra_id = compras.id
LEFT JOIN PJ AS PJIN ON cnpj_instituicao = PJIN.cnpj
LEFT JOIN PJ AS PJFO ON cnpj_fornecedor = PJFO.cnpj
LEFT JOIN PJ AS PJFA ON cnpj_fabricante = PJFA.cnpj
order by data desc);

""")
conn.close()


In [14]:
# conn = duckdb.connect('consolidado.db')


# conn.execute("""
# CREATE VIEW ITENSCOMPRA AS
# (SELECT
#     compras.data,
#     compras.cnpj_instituicao,
#     PJIN.nome as instituicao,
#     PJIN.municipio,
#     PJIN.UF,
#     compras.modalidade,
#     compras.tipo,
#     itens.licitacao,
#     materials.* exclude (id),
#     itens.cnpj_fornecedor,
#     PJFO.nome as fornecedor,
#     itens.NF,
#     itens.cnpj_fabricante,
#     PJFA.nome as fabricante,
#     itens.anvisa,
#     itens.qtde,
#     itens.unitario
# FROM itens
# LEFT JOIN materials on itens.materials_id = materials.id
# LEFT JOIN compras ON itens.compra_id = compras.id
# LEFT JOIN PJ AS PJIN ON cnpj_instituicao = PJIN.cnpj
# LEFT JOIN PJ AS PJFO ON cnpj_fornecedor = PJFO.cnpj
# LEFT JOIN PJ AS PJFA ON cnpj_fabricante = PJFA.cnpj
# order by data desc);

# """)
# conn.close()


In [15]:
!rm *.xlsx*

In [16]:
# select itens.*, exclude (codigo, unidade), materials.id as materials_id from itens left join materials on itens.codigo=materials.codigobr and itens.unidade = materials.unidade;

In [ ]:
insert into Item
select id, compra_id, anvisa, cnpj_fornecedor, cnpj_fabricante, qtde, unitario, licitacao, NF, materials_id as material_id
from consolidado.Itens

INSERT INTO Compra (id, data, modalidade, tipo, cnpj_instituicao)
SELECT
    id,
    CAST(data AS TIMESTAMP), -- Casting DATE to TIMESTAMP
    modalidade,
    tipo,
    cnpj_instituicao
FROM
    consolidado.Compras;

insert into Material  select id, codigobr, unidade, classe, pdm, descricao, disponivel from consolidado.Materials;

In [17]:
# CREATE VIEW ITENSCOMPRA AS
# (SELECT * FROM ITENS LEFT JOIN (select id as compra_id, data, cnpj_instituicao, modalidade, tipo from COMPRAS) as c ON ITENS.compra_id = c.compra_id)
# from itens join compras on compra_id = id